# How to create files for Ami's pipeline

In [ ]:
# Importing the Libraries
import os
import platform
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle
from itertools import product
from scipy.stats import skew, kurtosis, pearsonr, iqr, zscore
from scipy.signal import butter, welch, filtfilt, resample
import time
import re
import copy
# import nolds
from sklearn import preprocessing
import scipy.io # test matlab code

# from PreprocessFunctions import * # functions for unstacked df with raw data
# from PreprocessFcns import *

%matplotlib inline

In [ ]:
SubID = 'HC03'
testpath31 = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data\biostamp_data\controls'
testpath32 = os.path.join(testpath31, SubID)
testactdict, testtimestamp = extract_data(SubID, testpath32)
testunstacked3 = unstack_ar_dictionary(testactdict)

In [242]:
test = testunstacked3.query('((location=="sacrum")or(location=="distal_lateral_shank_left")or \
                         (location=="distal_lateral_shank_right")) & \
                     sensor=="accel" & \
                    ( (task=="WALKING") or (task=="STAIRS DOWN") or (task=="STAIRS UP") )&\
                     ((trial==0) or (trial==1)) ')
test

,task,trial,location,sensor,rawdata
684,WALKING,0,distal_lateral_shank_left,accel,Accel X (g) Accel Y (g) Acce...
687,WALKING,0,distal_lateral_shank_right,accel,Accel X (g) Accel Y (g) Acce...
711,WALKING,0,sacrum,accel,Accel X (g) Accel Y (g) Acce...
732,WALKING,1,distal_lateral_shank_left,accel,Accel X (g) Accel Y (g) Acce...
735,WALKING,1,distal_lateral_shank_right,accel,Accel X (g) Accel Y (g) Acce...
759,WALKING,1,sacrum,accel,Accel X (g) Accel Y (g) Acce...
876,STAIRS DOWN,0,distal_lateral_shank_left,accel,Accel X (g) Accel Y (g) Acce...
879,STAIRS DOWN,0,distal_lateral_shank_right,accel,Accel X (g) Accel Y (g) Acce...
903,STAIRS DOWN,0,sacrum,accel,Accel X (g) Accel Y (g) Acce...
924,STAIRS DOWN,1,distal_lateral_shank_left,accel,Accel X (g) Accel Y (g) Acce...


In [256]:
# save rawdata as separate files
amipath = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition'
test.rawdata[684].to_csv(os.path.join(amipath, 'walking_leftankle.csv'), index=False)
test.rawdata[687].to_csv(os.path.join(amipath, 'walking_rightankle.csv'), index=False)
test.rawdata[711].to_csv(os.path.join(amipath, 'walking_sacrum.csv'), index=False)
test.rawdata[876].to_csv(os.path.join(amipath, 'stairdn_leftankle.csv'), index=False)
test.rawdata[879].to_csv(os.path.join(amipath, 'stairdn_rightankle.csv'), index=False)
test.rawdata[903].to_csv(os.path.join(amipath, 'stairdn_sacrum.csv'), index=False)
test.rawdata[972].to_csv(os.path.join(amipath, 'stairup_leftankle.csv'), index=False)
test.rawdata[975].to_csv(os.path.join(amipath, 'stairup_rightankle.csv'), index=False)
test.rawdata[999].to_csv(os.path.join(amipath, 'stairup_sacrum.csv'), index=False)

In [243]:
# matlab doesn't like '_' characters
# test.location = test.location.str.replace('_', '')
test.location = test.location.str.replace('distal_lateral_shank_left', 'leftankle')
test.location = test.location.str.replace('distal_lateral_shank_right', 'rightankle')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [244]:
test.head(3)

,task,trial,location,sensor,rawdata
684,WALKING,0,leftankle,accel,Accel X (g) Accel Y (g) Acce...
687,WALKING,0,rightankle,accel,Accel X (g) Accel Y (g) Acce...
711,WALKING,0,sacrum,accel,Accel X (g) Accel Y (g) Acce...


In [127]:
amipath = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition'
test.to_hdf(os.path.join(amipath, 'testdata.h5'),key='df',mode='w')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['task', 'location', 'sensor', 'rawdata']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [245]:
walk3sensor = test.query('((location=="sacrum")or(location=="leftankle")or \
                         (location=="rightankle")) & \
                     sensor=="accel" & \
                    (task=="WALKING") &\
                     ((trial==0) or (trial==1)) ')
stairdown3sensor = test.query('((location=="sacrum")or(location=="leftankle")or \
                         (location=="rightankle")) & \
                     sensor=="accel" & \
                    (task=="STAIRS DOWN") &\
                     ((trial==0) or (trial==1)) ')
stairup3sensor = test.query('((location=="sacrum")or(location=="leftankle")or \
                         (location=="rightankle")) & \
                     sensor=="accel" & \
                    (task=="STAIRS UP") &\
                     ((trial==0) or (trial==1)) ')


In [246]:
test1 = walk3sensor[walk3sensor.trial==0][['location','rawdata']]
test1.index = [0,0,0]


In [247]:
test1 = test1.pivot(columns='location',values='rawdata')

In [248]:
test1.columns.name = None

In [249]:
test1

,leftankle,rightankle,sacrum
0,Accel X (g) Accel Y (g) Acce...,Accel X (g) Accel Y (g) Acce...,Accel X (g) Accel Y (g) Acce...


In [252]:
amipath = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition'
# test1.to_hdf(os.path.join(amipath, 'test1.h5'),key='df',mode='w')
# test1.to_csv(os.path.join(amipath, 'test1.csv'), index=False)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['leftankle', 'rightankle', 'sacrum']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [235]:
test1

,distal_lateral_shank_left,distal_lateral_shank_right,sacrum
0,Accel X (g) Accel Y (g) Acce...,Accel X (g) Accel Y (g) Acce...,Accel X (g) Accel Y (g) Acce...


In [192]:
walk3sensor[walk3sensor.trial==0][['location','rawdata']].\
pivot(columns='location',values='rawdata')

location,distal_lateral_shank_left,distal_lateral_shank_right,sacrum
684,Accel X (g) Accel Y (g) Acce...,NaN,NaN
687,NaN,Accel X (g) Accel Y (g) Acce...,NaN
711,NaN,NaN,Accel X (g) Accel Y (g) Acce...


In [193]:
walk3sensor[walk3sensor.trial==0][['location','rawdata']].index

Int64Index([684, 687, 711], dtype='int64')

In [130]:
walk_trial0 = walk3sensor[walk3sensor.trial==0][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
walk_trial1 = walk3sensor[walk3sensor.trial==1][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairdown_trial0 = stairdown3sensor[stairdown3sensor.trial==0][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairdown_trial1 = stairdown3sensor[stairdown3sensor.trial==1][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairup_trial0 = stairup3sensor[stairup3sensor.trial==0][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairup_trial1 = stairup3sensor[stairup3sensor.trial==1][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')

In [131]:
walk_trial0

location,distal_lateral_shank_left,distal_lateral_shank_right,sacrum
trial,,,
0,Accel X (g) Accel Y (g) Acce...,Accel X (g) Accel Y (g) Acce...,Accel X (g) Accel Y (g) Acce...


In [ ]:
test = testunstacked3.query('((location=="sacrum")or(location=="distal_lateral_shank_left")or \
                         (location=="distal_lateral_shank_right")) & \
                     sensor=="accel" & \
                    ( (task=="WALKING") or (task=="STAIRS DOWN") or (task=="STAIRS UP") )&\
                     ((trial==0) or (trial==1)) ')
test

# save rawdata as separate files
amipath = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition'
test.rawdata[684].to_csv(os.path.join(amipath, 'walking_leftankle.csv'), index=False)
test.rawdata[687].to_csv(os.path.join(amipath, 'walking_rightankle.csv'), index=False)
test.rawdata[711].to_csv(os.path.join(amipath, 'walking_sacrum.csv'), index=False)
test.rawdata[876].to_csv(os.path.join(amipath, 'stairdn_leftankle.csv'), index=False)
test.rawdata[879].to_csv(os.path.join(amipath, 'stairdn_rightankle.csv'), index=False)
test.rawdata[903].to_csv(os.path.join(amipath, 'stairdn_sacrum.csv'), index=False)
test.rawdata[972].to_csv(os.path.join(amipath, 'stairup_leftankle.csv'), index=False)
test.rawdata[975].to_csv(os.path.join(amipath, 'stairup_rightankle.csv'), index=False)
test.rawdata[999].to_csv(os.path.join(amipath, 'stairup_sacrum.csv'), index=False)

# matlab doesn't like '_' characters
# test.location = test.location.str.replace('_', '')
test.location = test.location.str.replace('distal_lateral_shank_left', 'leftankle')
test.location = test.location.str.replace('distal_lateral_shank_right', 'rightankle')

test.head(3)

amipath = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition'
test.to_hdf(os.path.join(amipath, 'testdata.h5'),key='df',mode='w')

walk3sensor = test.query('((location=="sacrum")or(location=="leftankle")or \
                         (location=="rightankle")) & \
                     sensor=="accel" & \
                    (task=="WALKING") &\
                     ((trial==0) or (trial==1)) ')
stairdown3sensor = test.query('((location=="sacrum")or(location=="leftankle")or \
                         (location=="rightankle")) & \
                     sensor=="accel" & \
                    (task=="STAIRS DOWN") &\
                     ((trial==0) or (trial==1)) ')
stairup3sensor = test.query('((location=="sacrum")or(location=="leftankle")or \
                         (location=="rightankle")) & \
                     sensor=="accel" & \
                    (task=="STAIRS UP") &\
                     ((trial==0) or (trial==1)) ')


test1 = walk3sensor[walk3sensor.trial==0][['location','rawdata']]
test1.index = [0,0,0]


test1 = test1.pivot(columns='location',values='rawdata')

test1.columns.name = None

test1

amipath = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition'
# test1.to_hdf(os.path.join(amipath, 'test1.h5'),key='df',mode='w')
# test1.to_csv(os.path.join(amipath, 'test1.csv'), index=False)

test1

walk3sensor[walk3sensor.trial==0][['location','rawdata']].\
pivot(columns='location',values='rawdata')

walk3sensor[walk3sensor.trial==0][['location','rawdata']].index

walk_trial0 = walk3sensor[walk3sensor.trial==0][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
walk_trial1 = walk3sensor[walk3sensor.trial==1][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairdown_trial0 = stairdown3sensor[stairdown3sensor.trial==0][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairdown_trial1 = stairdown3sensor[stairdown3sensor.trial==1][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairup_trial0 = stairup3sensor[stairup3sensor.trial==0][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')
stairup_trial1 = stairup3sensor[stairup3sensor.trial==1][['location','rawdata','trial']]. \
    pivot(index='trial',columns='location',values='rawdata')

walk_trial0